In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from src import utils, load_data
from src.models import lth_maml as lth
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.tensorboard import SummaryWriter
from collections import namedtuple
from src import utils

print(f"found gpu: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

found gpu: True


In [2]:
expr_params = {
    "prune_strategy": {
        "name": "global",
        "rate": .1,
        "iterations": 35,
    },
    "seeds": {
        "torch": 222,
        "cuda": 222,
        "numpy": 222
    },
    "expr_id": utils.new_expr_id("lth maml"),
    "model_training_params": {
        "training_iterations": 60000, #epoch
        "n_way": 5,                        # number of classes to choose between for each task
        "k_spt": 1,                        # k shot for support set (number of examples per class per task)
        "k_qry": 15,                       # k shot for query set (number of examples per class per task)
        "imgsz": 84,                       # image size
        "imgc": 3,                         # this isn't used anywhere????? no idea what it does???? they say its supposed to be 1 or 3...
        "task_num": 4,                     # meta model batch size
        "meta_lr": 1e-3,                   # meta model learning rate
        "update_lr": 0.01,                 # task specific model learning rate
        "update_step": 5,                  # task specific model training epochs
        "update_step_test": 10,            # task specific model testing epochs
#         "optimizer": ("adam", {"lr": 0.0001}),
#         "loss_func": "cross_entropy",
        "model_name": "MAML",
        "dataset_name": "mini_imagenet",
        "layer_definitions": None
    }
}

expr_params["model_training_params"]["layer_definitions"] = [
    ('conv2d', [32, 3, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [expr_params["model_training_params"]["n_way"], 32 * 5 * 5]) # 32 * 5 * 5
]
utils.set_seeds(expr_params["seeds"])

In [3]:
args = expr_params["model_training_params"]
dataset = load_data.mini_imagenet(args)

# train, val, test = dataset

# images_support, labels_support, images_query, labels_query = next(iter(train))
# print(f"num train batches: {len(train)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

# images = images_query.flatten(0, 1)
# labels = labels_query.flatten(0, 1)
# for i in range(len({i.numpy().item() for i in labels})):
#     grid = torchvision.utils.make_grid(images[labels == i])
#     with SummaryWriter(f'tensorboard/lth_maml_miniimagenet') as writer:
#         writer.add_image('processed', grid, i)
#         writer.flush()

# images_support, labels_support, images_query, labels_query = next(iter(val))
# print(f"num val batches: {len(val)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

# images_support, labels_support, images_query, labels_query = next(iter(test))
# print(f"num test batches: {len(test)}")
# print(f"images_support {images_support.shape}")
# print(f"labels_support {labels_support.shape}")
# print(f"images_query {images_query.shape}")
# print(f"labels_query {labels_query.shape}")

shuffle DB :train, b:10000, 5-way, 1-shot, 15-query, resize:84
shuffle DB :val, b:100, 5-way, 1-shot, 15-query, resize:84
shuffle DB :test, b:100, 5-way, 1-shot, 15-query, resize:84


In [4]:
print(f"starting lth run {expr_params['expr_id']}")
mask = lth.run(dataset, expr_params)

starting lth run expr 2020-11-05 | 16:09:19 lth maml E0jWH
starting prune iteration 0


train epoch 0


step: 0 	training acc: [0.20666667 0.22333333 0.22333333 0.23666667 0.23666667 0.23666667]
validating model...
val acc: [0.1919 0.2432 0.2449 0.2467 0.2476 0.247  0.2488 0.2494 0.2496 0.2496
 0.2502]

train epoch 1


step: 0 	training acc: [0.21       0.22666667 0.22666667 0.23666667 0.23333333 0.23333333]
validating model...
val acc: [0.1953 0.2578 0.264  0.2664 0.268  0.2683 0.268  0.2695 0.2695 0.27
 0.2693]

train epoch 2


step: 0 	training acc: [0.17666667 0.22333333 0.23       0.23       0.22666667 0.23333333]
validating model...
val acc: [0.1954 0.2593 0.2705 0.2751 0.2778 0.2788 0.281  0.2827 0.2842 0.2856
 0.2856]

0. perc_left: 1.0, test_acc 0.27197265625
starting prune iteration 1


...

step: 0 	training acc: [0.18666667 0.27666667 0.29       0.29333333 0.3        0.29666667]
validating model...
val acc: [0.2041 0.247  0.2532 0.258  0.259  0.2617 0.2634 0.265  0.266  0.2673
 0.268 ]

2. perc_left: 0.81, test_acc 0.267822265625
starting prune iteration 3


train epoch 0


step: 0 	training acc: [0.2        0.29666667 0.29666667 0.29333333 0.29333333 0.29333333]
validating model...
val acc: [0.2063 0.2466 0.2484 0.2512 0.2537 0.2537 0.255  0.2551 0.2554 0.256
 0.2563]

train epoch 1


step: 0 	training acc: [0.19666667 0.24666667 0.25333333 0.24666667 0.24333333 0.24666667]
validating model...



KeyboardInterrupt: 